## Benwin - D22032


## Text Classification: Problem 2 (using word2vec embeddings)

### Importing the required Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import nltk
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
from sklearn import tree

# For fitting Classification tree
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

import sklearn.ensemble as ensemble #import ensemble for bagging
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,f1_score #import Scoring metric
from sklearn.ensemble import RandomForestClassifier #import randomforest Classifier
from sklearn.ensemble import AdaBoostClassifier #import adaboostclassifer
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

import nltk
from gensim.models import KeyedVectors

### Upload and read the Tweet Data

In [2]:
tweets = pd.read_csv("tweets.csv")

In [3]:
tweets.head()

,Tweet,Avg
0,"I have to say, Apple has by far the best custo...",2.0
1,iOS 7 is so fricking smooth & beautiful!! #Tha...,2.0
2,LOVE U @APPLE,1.8
3,"Thank you @apple, loving my new iPhone 5S!!!!!...",1.8
4,.@apple has the best customer service. In and ...,1.8


### Test Pre-processing

#### Step 1. Converting the text into lower cases

In [4]:
#Convert a string into lower case
twt = tweets.Tweet[1]
twt.lower()

'ios 7 is so fricking smooth & beautiful!! #thanxapple @apple'

In [5]:
#normalizing all the tweets
for i in range(len(tweets.Tweet)):
    tweets.Tweet[i] = tweets.Tweet[i].lower()
tweets.head()

,Tweet,Avg
0,"i have to say, apple has by far the best custo...",2.0
1,ios 7 is so fricking smooth & beautiful!! #tha...,2.0
2,love u @apple,1.8
3,"thank you @apple, loving my new iphone 5s!!!!!...",1.8
4,.@apple has the best customer service. in and ...,1.8


#### Step 2. Remove Punctuations/special symbols  

In [6]:
#Removing punctuation from a single tweet
import string
p = string.punctuation
remv_punc = str.maketrans("", "", p)
a=twt.translate(remv_punc)
a

'iOS 7 is so fricking smooth  beautiful ThanxApple Apple'

In [7]:
#Removing punctuation from all the tweets
for i in range(len(tweets.Tweet)):
    tweets.Tweet[i] = tweets.Tweet[i].translate(remv_punc)

In [8]:
tweets

,Tweet,Avg
0,i have to say apple has by far the best custom...,2.0
1,ios 7 is so fricking smooth beautiful thanxap...,2.0
2,love u apple,1.8
3,thank you apple loving my new iphone 5s apple...,1.8
4,apple has the best customer service in and out...,1.8
...,...,...
1176,freak apple,-2.0
1177,why cant i freaking see pictures on my tl im a...,-2.0
1178,apple you freaking cows freak you,-2.0
1179,apple i hate you why is my phone not working i...,-2.0


In [9]:
# Subsetting the data where average of tweet = 0
tweets = tweets.loc[tweets.Avg!=0] 
tweets

,Tweet,Avg
0,i have to say apple has by far the best custom...,2.0
1,ios 7 is so fricking smooth beautiful thanxap...,2.0
2,love u apple,1.8
3,thank you apple loving my new iphone 5s apple...,1.8
4,apple has the best customer service in and out...,1.8
...,...,...
1176,freak apple,-2.0
1177,why cant i freaking see pictures on my tl im a...,-2.0
1178,apple you freaking cows freak you,-2.0
1179,apple i hate you why is my phone not working i...,-2.0


In [10]:
#Resetting the index of the dataframe and dropping the original index
tweets.reset_index(drop=True,inplace=True)
tweets

,Tweet,Avg
0,i have to say apple has by far the best custom...,2.0
1,ios 7 is so fricking smooth beautiful thanxap...,2.0
2,love u apple,1.8
3,thank you apple loving my new iphone 5s apple...,1.8
4,apple has the best customer service in and out...,1.8
...,...,...
839,freak apple,-2.0
840,why cant i freaking see pictures on my tl im a...,-2.0
841,apple you freaking cows freak you,-2.0
842,apple i hate you why is my phone not working i...,-2.0


#### Step 3. Removing Stopwords from all the tweets

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#NLTK stopword list
stop_words = stopwords.words("english")
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
len(stop_words)

179

In [14]:
#Adding the word apple to the list of stopwords
stop_words.append("apple")
len(stop_words)

180

In [15]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
#Remove stop words from a single tweet
" ".join([w for w in tweets.Tweet[0].split() if w not in stop_words])

'say far best customer care service ever received appstore'

In [17]:
[w for w in tweets.Tweet[0].split() if w not in stop_words]

['say',
 'far',
 'best',
 'customer',
 'care',
 'service',
 'ever',
 'received',
 'appstore']

In [18]:
len(tweets.Tweet)

844

In [19]:
#Removing stopwords from all the tweets
for i in range(len(tweets.Tweet)):
    tweets.Tweet[i] = " ".join([w for w in tweets.Tweet[i].split() if w not in stop_words])

In [20]:
tweets.head()

,Tweet,Avg
0,say far best customer care service ever receiv...,2.0
1,ios 7 fricking smooth beautiful thanxapple,2.0
2,love u,1.8
3,thank loving new iphone 5s iphone5s pictwitter...,1.8
4,best customer service new phone 10min,1.8


#### Step 4 . Remove white spaces

In [21]:
for i in range(len(tweets.Tweet)):
    tweets.Tweet[i] = tweets.Tweet[i].replace("  ", " ").strip()

In [22]:
tweets

,Tweet,Avg
0,say far best customer care service ever receiv...,2.0
1,ios 7 fricking smooth beautiful thanxapple,2.0
2,love u,1.8
3,thank loving new iphone 5s iphone5s pictwitter...,1.8
4,best customer service new phone 10min,1.8
...,...,...
839,freak,-2.0
840,cant freaking see pictures tl im annoyed freak...,-2.0
841,freaking cows freak,-2.0
842,hate phone working im going freak,-2.0


In [23]:
for i in range(len(tweets.Tweet)):
    tweets.Tweet[i] = " ".join([w for w in tweets.Tweet[i].split() if not w.isdigit()])
tweets

,Tweet,Avg
0,say far best customer care service ever receiv...,2.0
1,ios fricking smooth beautiful thanxapple,2.0
2,love u,1.8
3,thank loving new iphone 5s iphone5s pictwitter...,1.8
4,best customer service new phone 10min,1.8
...,...,...
839,freak,-2.0
840,cant freaking see pictures tl im annoyed freak...,-2.0
841,freaking cows freak,-2.0
842,hate phone working im going freak,-2.0


# Word2vec Embedding

In [24]:
embeddings = KeyedVectors.load_word2vec_format('C:\keniwn\GoogleNews-vectors-negative300.bin', binary = True)

### Get word embeddings of the our Words

In [25]:
embeddings.index_to_key

['</s>',
 'in',
 'for',
 'that',
 'is',
 'on',
 '##',
 'The',
 'with',
 'said',
 'was',
 'the',
 'at',
 'not',
 'as',
 'it',
 'be',
 'from',
 'by',
 'are',
 'I',
 'have',
 'he',
 'will',
 'has',
 '####',
 'his',
 'an',
 'this',
 'or',
 'their',
 'who',
 'they',
 'but',
 '$',
 'had',
 'year',
 'were',
 'we',
 'more',
 '###',
 'up',
 'been',
 'you',
 'its',
 'one',
 'about',
 'would',
 'which',
 'out',
 'can',
 'It',
 'all',
 'also',
 'two',
 'after',
 'first',
 'He',
 'do',
 'time',
 'than',
 'when',
 'We',
 'over',
 'last',
 'new',
 'other',
 'her',
 'people',
 'into',
 'In',
 'our',
 'there',
 'A',
 'she',
 'could',
 'just',
 'years',
 'some',
 'U.S.',
 'three',
 'million',
 'them',
 'what',
 'But',
 'so',
 'no',
 'like',
 'if',
 'only',
 'percent',
 'get',
 'did',
 'him',
 'game',
 'back',
 'because',
 'now',
 '#.#',
 'before',
 'company',
 'any',
 'team',
 'against',
 'off',
 'This',
 'most',
 'made',
 'through',
 'make',
 'second',
 'state',
 'well',
 'day',
 'season',
 'says',
 'w

In [26]:
def vec(a):
    return(embeddings[a])

In [27]:
# Converting each document into a vector
dictionary = {}
index_list = []
for tweet,index in zip(tweets.Tweet,tweets.index):
    rev_list = []
    for word in tweet.split():
        if word in embeddings.index_to_key:
            index_list.append(index)
            rev_list.append(vec(word))
    dictionary[tweet] = np.sum(np.array(rev_list),axis=0)
index_set = set(index_list)

In [41]:
document_matrix = pd.DataFrame(dictionary).T
document_matrix

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
say far best customer care service ever received appstore,-0.493164,-0.095337,0.296143,0.540649,-0.170532,0.215515,0.673096,-0.081055,1.091064,0.981018,...,-0.279327,0.084229,-1.071533,0.579533,0.664551,0.483276,-0.215881,-0.071289,0.313599,-0.570312
ios fricking smooth beautiful thanxapple,0.127014,-0.030273,0.273621,-0.040039,-0.403809,-0.032959,0.408875,-0.386108,0.112793,0.597656,...,-0.307495,0.284668,-0.612305,-0.289963,-0.227783,0.101074,-0.539062,-0.494812,-0.041748,0.389404
love u,-0.150879,-0.105713,0.189941,0.146729,-0.448242,-0.060059,0.079102,-0.544922,-0.199707,0.302246,...,0.082275,0.326172,-0.358398,-0.585938,-0.205322,-0.469727,-0.130859,-0.369141,-0.176514,0.253418
thank loving new iphone 5s iphone5s pictwittercomxmhjcu4pcb,-0.685974,-0.154663,0.053223,0.654785,-0.353271,-0.230469,0.068115,-0.525146,-0.131836,0.474243,...,0.090332,0.766357,-0.982422,-0.664551,-0.309082,-0.283630,-0.598259,0.078613,-0.448792,0.520508
best customer service new phone 10min,-0.210693,0.010620,0.281250,0.090088,0.067139,-0.029053,-0.178711,0.016846,0.424316,0.662659,...,0.328705,0.305664,-0.824219,0.041870,0.265381,0.335083,-0.138672,0.325439,-0.018555,-0.471680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
freak u,-0.048828,0.039551,0.169617,0.110596,-0.368652,0.201172,-0.050781,-0.541016,-0.139648,-0.018066,...,0.467773,0.381836,-0.319824,-0.239258,-0.363281,-0.510742,-0.338379,-0.149780,-0.075073,0.107422
cant freaking see pictures tl im annoyed freak twitter,0.735931,0.201782,0.265518,1.291504,-1.499268,0.956970,0.374115,-1.322784,0.530029,0.329895,...,0.783691,1.047729,-0.649292,-0.248840,-1.650391,-1.156738,-0.439453,-0.505722,-0.418335,0.028564
freaking cows freak,0.586914,-0.016846,0.001648,0.423340,-0.325684,0.740234,0.166382,-0.296600,0.048584,-0.116699,...,0.114258,0.328918,-0.202637,0.109863,-0.231445,-0.142578,-0.358887,0.240845,0.085541,-0.060547
hate phone working im going freak,0.347351,0.309204,0.310913,0.426514,-0.717896,0.939087,0.136658,-0.739258,0.257324,0.149170,...,-0.155029,0.922363,-0.669189,-0.155579,-0.644287,-0.572754,-0.270996,-0.055420,0.218994,-0.158936


In [42]:
# There are duplicate reviews in my dataset
tweets.Tweet.nunique()

827

In [52]:
document_matrix=document_matrix.reset_index()
document_matrix

,level_0,index,0,1,2,3,4,5,6,7,...,291,292,293,294,295,296,297,298,299,Avg
0,0,say far best customer care service ever receiv...,-0.493164,-0.095337,0.296143,0.540649,-0.170532,0.215515,0.673096,-0.081055,...,0.084229,-1.071533,0.579533,0.664551,0.483276,-0.215881,-0.071289,0.313599,-0.570312,NaN
1,1,ios fricking smooth beautiful thanxapple,0.127014,-0.030273,0.273621,-0.040039,-0.403809,-0.032959,0.408875,-0.386108,...,0.284668,-0.612305,-0.289963,-0.227783,0.101074,-0.539062,-0.494812,-0.041748,0.389404,NaN
2,2,love u,-0.150879,-0.105713,0.189941,0.146729,-0.448242,-0.060059,0.079102,-0.544922,...,0.326172,-0.358398,-0.585938,-0.205322,-0.469727,-0.130859,-0.369141,-0.176514,0.253418,NaN
3,3,thank loving new iphone 5s iphone5s pictwitter...,-0.685974,-0.154663,0.053223,0.654785,-0.353271,-0.230469,0.068115,-0.525146,...,0.766357,-0.982422,-0.664551,-0.309082,-0.283630,-0.598259,0.078613,-0.448792,0.520508,NaN
4,4,best customer service new phone 10min,-0.210693,0.010620,0.281250,0.090088,0.067139,-0.029053,-0.178711,0.016846,...,0.305664,-0.824219,0.041870,0.265381,0.335083,-0.138672,0.325439,-0.018555,-0.471680,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,822,freak u,-0.048828,0.039551,0.169617,0.110596,-0.368652,0.201172,-0.050781,-0.541016,...,0.381836,-0.319824,-0.239258,-0.363281,-0.510742,-0.338379,-0.149780,-0.075073,0.107422,NaN
823,823,cant freaking see pictures tl im annoyed freak...,0.735931,0.201782,0.265518,1.291504,-1.499268,0.956970,0.374115,-1.322784,...,1.047729,-0.649292,-0.248840,-1.650391,-1.156738,-0.439453,-0.505722,-0.418335,0.028564,NaN
824,824,freaking cows freak,0.586914,-0.016846,0.001648,0.423340,-0.325684,0.740234,0.166382,-0.296600,...,0.328918,-0.202637,0.109863,-0.231445,-0.142578,-0.358887,0.240845,0.085541,-0.060547,NaN
825,825,hate phone working im going freak,0.347351,0.309204,0.310913,0.426514,-0.717896,0.939087,0.136658,-0.739258,...,0.922363,-0.669189,-0.155579,-0.644287,-0.572754,-0.270996,-0.055420,0.218994,-0.158936,NaN


In [53]:
document_matrix['Avg']=np.nan
document_matrix['Avg']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
822   NaN
823   NaN
824   NaN
825   NaN
826   NaN
Name: Avg, Length: 827, dtype: float64

In [54]:
for i in range(len(tweets.Tweet)):
    for j in range(len(document_matrix)):
        if document_matrix['index'][j]==tweets['Tweet'][i]:
            document_matrix['Avg'][j]=tweets['Avg'][i]

In [55]:
document_matrix.set_index('index',inplace=True)
y = document_matrix.Avg
y

index
say far best customer care service ever received appstore      2.0
ios fricking smooth beautiful thanxapple                       2.0
love u                                                         1.8
thank loving new iphone 5s iphone5s pictwittercomxmhjcu4pcb    1.8
best customer service new phone 10min                          1.8
                                                              ... 
freak u                                                       -2.0
cant freaking see pictures tl im annoyed freak twitter        -2.0
freaking cows freak                                           -2.0
hate phone working im going freak                             -2.0
agounalakis thats nasty nasty brat                            -2.0
Name: Avg, Length: 827, dtype: float64

In [56]:
#df = document_matrix.iloc[:,:-1]

# Classification Model

In [57]:
df.shape

(827, 302)

In [60]:
avg=document_matrix.Avg[document_matrix.Avg!=0]
avg.value_counts()
avg

index
say far best customer care service ever received appstore      2.0
ios fricking smooth beautiful thanxapple                       2.0
love u                                                         1.8
thank loving new iphone 5s iphone5s pictwittercomxmhjcu4pcb    1.8
best customer service new phone 10min                          1.8
                                                              ... 
freak u                                                       -2.0
cant freaking see pictures tl im annoyed freak twitter        -2.0
freaking cows freak                                           -2.0
hate phone working im going freak                             -2.0
agounalakis thats nasty nasty brat                            -2.0
Name: Avg, Length: 827, dtype: float64

### Creating Target Variable y as binary classification

In [61]:
#Creating a target variable
y = avg <= 0
y

index
say far best customer care service ever received appstore      False
ios fricking smooth beautiful thanxapple                       False
love u                                                         False
thank loving new iphone 5s iphone5s pictwittercomxmhjcu4pcb    False
best customer service new phone 10min                          False
                                                               ...  
freak u                                                         True
cant freaking see pictures tl im annoyed freak twitter          True
freaking cows freak                                             True
hate phone working im going freak                               True
agounalakis thats nasty nasty brat                              True
Name: Avg, Length: 827, dtype: bool

In [78]:
df.isnull().sum()

index      0
0          0
1          0
2          0
3          0
        ... 
296        0
297        0
298        0
299        0
Avg      827
Length: 302, dtype: int64

In [79]:
W2V = df.copy()

In [80]:
W2V = W2V.drop(["Avg"],axis=1)

# Train-Validation-Test Split

In [81]:
#For train-test split
from sklearn.model_selection import train_test_split   #import train_test_split function

X_train, X_test, y_train, y_test = train_test_split(W2V, y, test_size = 0.20)  # X-predictors, y-target

#Train-Validation Split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.20)

## Decision Tree Model

In [82]:
#Training the model with best hyperparameters
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(X_train,y_train)
pred = dt.predict(X_valid)

pred = dt.predict(X_valid)
#y_pred #predictions done by decision tree

ValueError: could not convert string to float: 'smh rt cemorecake718 rt swaggal worst rt matiucurvegawd yooo iphone battery sus'

In [ ]:
print("Accuracy_score of Decision Tree:",accuracy_score(y_valid,pred)) # accuracy score of Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred)) # f1 score of Decision tree

In [ ]:
#for plotting a decision tree
from sklearn import tree

fig, axes = plt.subplots(figsize =(3,3), dpi=300)

tree.plot_tree(dt, feature_names = df.columns, filled = False)
plt.show()

## Fully Growned Decision Tree Model

In [ ]:
#By default, the Decision Tree function doesn’t perform any pruning and allows the tree to grow as much as it can.
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
pred = dt.predict(X_valid)

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred)
cm

In [ ]:
print("Accuracy_score of fully growned Decision Tree:",accuracy_score(y_valid,pred)) # accuracy score of fully growned Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

## Pruned Decision Tree Model

In [83]:
#pruning-Finding alpha
path = dt.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas
ccp_alphas

ValueError: could not convert string to float: 'smh rt cemorecake718 rt swaggal worst rt matiucurvegawd yooo iphone battery sus'

In [ ]:
train_accuracy = {}         #Dictionary to store alpha and mae
validation_accuracy = {}

for i in ccp_alphas:
    dt = DecisionTreeClassifier(ccp_alpha=i)
    dt.fit(X_train,y_train)
    pred_valid = dt.predict(X_valid)
    pred_train = dt.predict(X_train)

    train_accuracy[i] = accuracy_score(y_train,pred_train)
    validation_accuracy[i] = accuracy_score(y_valid,pred_valid)

opt_ccp = max(validation_accuracy,key = validation_accuracy.get) #Optimum Cost Complexity Parameter
opt_ccp #optimal CCP_alphas 

In [ ]:
prun_dt = DecisionTreeClassifier(ccp_alpha=opt_ccp) #Fitting pruned decision tree model using optimum alpha
prun_dt.fit(X_train,y_train)

pred = prun_dt.predict(X_valid)

In [84]:
print("Accuracy_score of Pruned Decision Tree:",accuracy_score(y_valid,pred)) # accuracy score of fully growned Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

NameError: name 'pred' is not defined

## Bagged Decision Tree Model

In [85]:
dt = DecisionTreeClassifier()      #decision tree model as base classifier for bagging -ensemble

baggedModel = ensemble.BaggingClassifier(base_estimator=dt) #bagging with DT models

baggedModel.fit(X_train,y_train) #fitting training data in bagged model
pred_bg = baggedModel.predict(X_valid)

ValueError: could not convert string to float: 'smh rt cemorecake718 rt swaggal worst rt matiucurvegawd yooo iphone battery sus'

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_bg)
cm

In [ ]:
print("Accuracy_score of Bagged Decision Tree:",accuracy_score(y_valid,pred_bg)) #accuracy of bagged decision tree model
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

## Random Forest Model

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)        # fitting training data in random forest model

pred_rf= rf.predict(X_valid)

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_rf)
cm

In [86]:
print("Accuracy_score of Random Forest:",accuracy_score(y_valid,pred_rf)) #accuracy of RandomForest model
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

NameError: name 'pred_rf' is not defined

## Adaboost Model

In [87]:
ada_boost = AdaBoostClassifier()
ada_boost.fit(X_train, y_train) # fitting training data in adaboost model

pred_ada = ada_boost.predict(X_valid) 

ValueError: could not convert string to float: 'smh rt cemorecake718 rt swaggal worst rt matiucurvegawd yooo iphone battery sus'

In [ ]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_ada)
cm

In [ ]:
print("Accuracy_score of Adaboost:",accuracy_score(y_valid,pred_ada)) #accuracy of adaboost model
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

In [88]:
#print("Recall_score of Adaboost:",recall_score(y_valid,pred_ada))

In [89]:
pd.DataFrame({'Decision tree with depth =5': [0.62,0.69], 'Fully Growned DT': [0.51,0.60], 'Pruned DT': [0.66,0.79], 'Bagged DT': [0.65,0.79], 'Random Forest': [0.72,0.79], 'Adaboost': [0.66,0.79] }, index=['Accuracy','F1_score'])

,Decision tree with depth =5,Fully Growned DT,Pruned DT,Bagged DT,Random Forest,Adaboost
Accuracy,0.62,0.51,0.66,0.65,0.72,0.66
F1_score,0.69,0.60,0.79,0.79,0.79,0.79


--------------------------------------------------------------------------------------------------------------------------------

# 2. TF-IDF Method

### Document-Term Matrix

#### 1. Get the Bag-Of-Words (BOW) Dataframe with TF-IDF vectorizor

In [90]:
tweets

,Tweet,Avg
0,say far best customer care service ever receiv...,2.0
1,ios fricking smooth beautiful thanxapple,2.0
2,love u,1.8
3,thank loving new iphone 5s iphone5s pictwitter...,1.8
4,best customer service new phone 10min,1.8
...,...,...
839,freak,-2.0
840,cant freaking see pictures tl im annoyed freak...,-2.0
841,freaking cows freak,-2.0
842,hate phone working im going freak,-2.0


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(tweets.Tweet)

names = tfidf_vectorizer.get_feature_names()

tfidf_vectors = tfidf_vectors.toarray()
tfidf_vectors = pd.DataFrame(tfidf_vectors, columns=names)
tfidf_vectors

,10min,13apple,16gb,16gbs,18th,1am,1jazzyjeff,1st,1u,2000ad,...,yikes,yldthng,yo,yooo,youd,youre,youve,z10,zimmerman,zippos
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.558987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
841,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
842,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2. Dimension Reduction

In [92]:
#using count vectoroizer to create a document-term matrix
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(tweets.Tweet).toarray()

In [93]:
#Document-Term Matrix
DTM = pd.DataFrame(X, columns=cv.get_feature_names())

#Remove terms that is conatined in less than 1% of the documents
for col in list(DTM):
    prop = DTM[col].sum()/DTM.shape[0]
    if prop*100 < 1:
        DTM = DTM.drop([col], axis=1)
        
selected_variables = list(DTM)

In [94]:
DTM.shape

(844, 140)

#### 3. Train-Validation-Test Split

In [95]:
#Creating the target variable
y = tweets.Avg <= 0
tfidf_vectors = tfidf_vectors[selected_variables]

#For test-train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_vectors, y, test_size = 0.2, stratify=y)

#Train-Validation Split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.25, stratify=y_train)

X_valid.head(5)

,5c,5s,already,amazon,android,anyone,app,apples,apps,back,...,well,wont,work,would,wow,wtf,yall,year,yet,youre
69,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
574,0.0,0.207132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
165,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
599,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
696,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240927,0.0,0.0


### Classification Model

#### 2. Classification Tree

In [96]:
#Classification Tree Model Fitting
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(X_train,y_train)
pred = dt.predict(X_valid)

pred = dt.predict(X_valid)

In [97]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred)
cm

array([[  9,  52],
       [  4, 104]], dtype=int64)

In [98]:
# accuracy score of Decision tree
print("Accuracy_score of Decision Tree with Depth=5:",accuracy_score(y_valid,pred)) 
# f1 score of Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred)) 

Accuracy_score of Decision Tree with Depth=5: 0.6686390532544378
F1_score of Decision Tree: 0.7878787878787878


## Fully Growned Decision Tree Model

In [99]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
pred = dt.predict(X_valid)

In [100]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred)
cm

array([[27, 34],
       [37, 71]], dtype=int64)

In [101]:
print("Accuracy_score of fully growned Decision Tree:",accuracy_score(y_valid,pred)) # accuracy score of fully growned Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

Accuracy_score of fully growned Decision Tree: 0.5798816568047337
F1_score of Decision Tree: 0.6666666666666667


## Pruned Decision Tree Model

In [102]:
#pruning-Finding alpha
path = dt.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas
ccp_alphas

array([0.        , 0.00058955, 0.0007017 , 0.00081044, 0.00098814,
       0.00118886, 0.00141543, 0.0017567 , 0.00177866, 0.00180443,
       0.00190309, 0.00197628, 0.00251923, 0.0025552 , 0.00258769,
       0.00263505, 0.00263505, 0.00270665, 0.00282326, 0.00296443,
       0.00296443, 0.00296443, 0.00296443, 0.00316206, 0.00316206,
       0.00316206, 0.00316206, 0.00329381, 0.00338792, 0.00341244,
       0.00355731, 0.00361378, 0.00363775, 0.00368906, 0.00372678,
       0.00375494, 0.00402053, 0.00441587, 0.0058182 , 0.00691872,
       0.00747362, 0.00813023, 0.01073634, 0.01113556, 0.01218616])

In [103]:
#Dictionary to store alpha and mae
train_accuracy = {}         
validation_accuracy = {}

for i in ccp_alphas:
    dt = DecisionTreeClassifier(ccp_alpha=i)
    dt.fit(X_train,y_train)
    pred_valid = dt.predict(X_valid)
    pred_train = dt.predict(X_train)

    train_accuracy[i] = accuracy_score(y_train,pred_train)
    validation_accuracy[i] = accuracy_score(y_valid,pred_valid)

#Optimum Cost Complexity Parameter    
opt_ccp = max(validation_accuracy,key = validation_accuracy.get) 
#optimal CCP_alphas 
opt_ccp 

0.0037549407114624497

In [104]:
#Fitting pruned decision tree model using optimum alpha
prun_dt = DecisionTreeClassifier(ccp_alpha=opt_ccp) 
prun_dt.fit(X_train,y_train)

pred = prun_dt.predict(X_valid)

In [105]:
print("Accuracy_score of Pruned Decision Tree:",accuracy_score(y_valid,pred)) # accuracy score of fully growned Decision tree
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

Accuracy_score of Pruned Decision Tree: 0.6804733727810651
F1_score of Decision Tree: 0.7804878048780488


## Bagged Decision Tree Model

In [106]:
#decision tree model as base classifier for bagging -ensemble
dt = DecisionTreeClassifier()      

#bagging with DT models
baggedModel = ensemble.BaggingClassifier(base_estimator=dt) 

#fitting training data in bagged model
baggedModel.fit(X_train,y_train) 
pred_bg = baggedModel.predict(X_valid)

In [107]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_bg)
cm

array([[33, 28],
       [41, 67]], dtype=int64)

In [108]:
#accuracy of bagged decision tree model
print("Accuracy_score of Bagged Decision Tree:",accuracy_score(y_valid,pred_bg)) 
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

Accuracy_score of Bagged Decision Tree: 0.591715976331361
F1_score of Decision Tree: 0.7804878048780488


## Random Forest Model

In [109]:
rf = RandomForestClassifier()
# fitting training data in random forest model
rf.fit(X_train, y_train)        

pred_rf= rf.predict(X_valid)

In [110]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_rf)
cm

array([[31, 30],
       [38, 70]], dtype=int64)

In [111]:
#accuracy of RandomForest model
print("Accuracy_score of Random Forest:",accuracy_score(y_valid,pred_rf)) 
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

Accuracy_score of Random Forest: 0.5976331360946746
F1_score of Decision Tree: 0.7804878048780488


## Adaboost Model

In [112]:
ada_boost = AdaBoostClassifier()
# fitting training data in adaboost model
ada_boost.fit(X_train, y_train) 

pred_ada = ada_boost.predict(X_valid) 

In [113]:
# Confusion Matrix 
cm = confusion_matrix(y_valid,pred_ada)
cm

array([[22, 39],
       [17, 91]], dtype=int64)

In [114]:
#accuracy of adaboost model
print("Accuracy_score of Adaboost:",accuracy_score(y_valid,pred_ada)) 
print("F1_score of Decision Tree:",f1_score(y_valid,pred))

Accuracy_score of Adaboost: 0.6686390532544378
F1_score of Decision Tree: 0.7804878048780488


# TF-IDF Results

In [115]:
pd.DataFrame({'Decision tree with depth =5': [0.64,0.76], 'Fully Growned DT': [0.61,0.70], 'Pruned DT': [0.64,0.76], 'Bagged DT': [0.65,0.76], 'Random Forest': [0.65,0.70], 'Adaboost': [0.64,0.70] }, index=['Accuracy','F1_score'])

,Decision tree with depth =5,Fully Growned DT,Pruned DT,Bagged DT,Random Forest,Adaboost
Accuracy,0.64,0.61,0.64,0.65,0.65,0.64
F1_score,0.76,0.70,0.76,0.76,0.70,0.70
